Notebook to load data from JSON files, connect and load data to MySQL
- Adding traditional stats for players and teams (two tables)

In [1]:
import mysql.connector as connector
from mysql.connector import Error, IntegrityError, InterfaceError
from tqdm import tqdm
from dotenv import load_dotenv
import os
import json
import time

In [ ]:
load_dotenv()
pw = os.getenv('mysql_pw')
connection = connector.connect(
        host='localhost',
        user='root',
        password=pw,
        port=3306
)

In [ ]:
cursor = connection.cursor()

In [ ]:
cursor.execute('USE nba_traditional')

In [ ]:
# Create query to insert teams into database
insert_team_query = """
INSERT INTO teams (team_id,team_abbrev,team_name,season,gp,wins,losses,win_pct,minutes,fgm,fga,fg_pct,
fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,blka,pf,pfd,pts,plus_minus)
VALUES (%(team_id)s, %(abbreviation)s, %(team_name)s, %(season)s, %(gp)s, %(wins)s, %(losses)s,
%(win_pct)s, %(minutes)s, %(fgm)s, %(fga)s, %(fg_pct)s, %(fg3m)s, %(fg3a)s, %(fg3_pct)s,
%(ftm)s, %(fta)s, %(ft_pct)s, %(oreb)s, %(dreb)s, %(reb)s, %(ast)s, %(tov)s, %(stl)s,
%(blk)s, %(blka)s, %(pf)s, %(pfd)s, %(pts)s, %(plus_minus)s)"""

In [ ]:
# Load TEAM data from JSON into the database
with open('/Files/team_data.json', 'r') as f:
    team_data = json.load(f)
    
for team in team_data:
    cursor.execute(insert_team_query, team_data[team])
    time.sleep(1)

In [ ]:
# Create query to insert players into database
insert_player_query = """
INSERT INTO players (player_id,team_id,player_name,season,gp,wins,losses,win_pct,minutes,fgm,fga,fg_pct,
fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,blka,pf,pfd,pts,plus_minus)
VALUES (%(player_id)s, %(team_id)s, %(player_name)s, %(season)s, %(gp)s, %(wins)s, %(losses)s,
%(win_pct)s, %(minutes)s, %(fgm)s, %(fga)s, %(fg_pct)s, %(fg3m)s, %(fg3a)s, %(fg3_pct)s,
%(ftm)s, %(fta)s, %(ft_pct)s, %(oreb)s, %(dreb)s, %(reb)s, %(ast)s, %(tov)s, %(stl)s,
%(blk)s, %(blka)s, %(pf)s, %(pfd)s, %(pts)s, %(plus_minus)s)"""

In [ ]:
# Load player data from JSON into the database

with open('/Files/player_data.json', 'r') as f:
    player_data = json.load(f)

for player in tqdm(player_data, desc='Uploading players...'):
    try:
        cursor.execute(insert_player_query, player_data[player])
        time.sleep(0.5)
    # Error checking
    except Error as e:
        print("General MySQL error: ", e)
        print(f'The player was: {player}')
    except InterfaceError as e:
        print("Interface error: ", e)
        print(f'The player was: {player}')
    except IntegrityError as e:
        print("Integrity error: ", e)
        print(f'The player was: {player}')

print('Complete')

In [ ]:
# Test the outcome of the player query
select_from_players_query = """SELECT * FROM players"""
cursor.execute(select_from_players_query)
players = cursor.fetchall()

for player in players:
    print(player)

In [ ]:
# Commit the connection
connection.commit()

In [10]:
# Close the connection
connection.close()